## 情感极性指标计算&异常预测查看

In [224]:
from sklearn.metrics import precision_recall_fscore_support
# from utils_squad_multi_plus import read_ecom_examples as cc
# from utils_squad_multi_plus import TRANS_SUBTYPE
from utils_skincare import read_ecom_examples as cc
from utils_skincare import TRANS_SUBTYPE

import os
import json
import numpy as np
import pandas as pd



def calc_polar_metric(dataset, preds_polar, preds_op,preds_as,need_print=False):
    polar_preds = []
    polar_labels = []
    polar_map={
        '正面':5,
        '负面':1,
        "中性":3
    }
    wrong = 0
    raw_datas = []
    for d in dataset:
        doc_tokens = d.doc_tokens.split(' ',1)[-1]
        
        polar_pred = preds_polar[d.qas_id].replace(" ","")
        polar_label = d.polar_answer_text
        polar_preds.append(polar_map.get(polar_pred, 0))
        polar_labels.append(polar_map.get(polar_label,0))
        
        opinion_pred = preds_op[d.qas_id].replace(" ","")
        opinion_label = d.op_answer_text
        raw_datas.append((d.question_text, doc_tokens, polar_label, polar_pred, opinion_label, opinion_pred))
        if need_print and  polar_label != polar_pred:
#         if need_print and d.orig_answer_text != preds_as[d.qas_id].replace(" ","") and preds_as[d.qas_id].replace(" ","")=="": 
            wrong += 1
            print(d.doc_tokens)
            print('label', 'aspect', d.orig_answer_text, 'opinion', opinion_label, 'polar', polar_label)
            print('predict','aspect', preds_as[d.qas_id].replace(" ",""), 'opinion', opinion_pred ,
                 'polar', polar_pred)
            print(f'***********************{wrong}********************************')


    s = precision_recall_fscore_support(polar_labels, polar_preds)
    df_temp = pd.DataFrame(np.array(s).T, columns=['prec', 'recall', 'f1','support'])
    df_temp['subtype'] = subtype
    
    df_temp['polarity'] = [0,1,3,5] if len(df_temp)==4 else [0,1,5]
    df_raw = pd.DataFrame(raw_datas, columns=['subtype', 'text', 'label','predict','op_label','op_pred'])
    return df_temp, df_raw



###  替换为待评估的品类
data_dir='skincare_v2'

target_dir = f"/data/projects/bert_pytorch/{data_dir}_out/all_in_one_v2/checkpoint-3000"
need_print=False


df_temp_polar = pd.DataFrame()
df_temp_raw = pd.DataFrame()
dirlist= os.listdir(target_dir)
# dirlist = ['Greasy']
for dirname in dirlist:
    subtype = dirname.replace('_'," ").replace('.','/')
    if  subtype in TRANS_SUBTYPE:
        polar_pred_path = os.path.join(target_dir, dirname, 'polar_predictions_.json')
        opinion_pred_path = os.path.join(target_dir, dirname, 'op_predictions_.json')
        aspect_pred_path = os.path.join(target_dir, dirname, 'predictions_.json')

        label_path = f'/data/projects/bert_pytorch/{data_dir}/{dirname}/dev.json'
        dataset = cc(label_path, True, subtype)
        preds_polar = json.load(open(polar_pred_path, 'r'))
        preds_op = json.load(open(opinion_pred_path, 'r'))
        preds_as = json.load(open(aspect_pred_path, 'r'))
        df_temp, df_raw  = calc_polar_metric(dataset, 
                                            preds_polar,
                                            preds_op,
                                            preds_as, 
                                            need_print=need_print)
        df_temp_polar = pd.concat([df_temp_polar, df_temp])
        df_temp_raw = pd.concat([df_temp_raw, df_raw])
        
# df_temp_polar.groupby('subtype').size()
df_temp_polar['weight_f1']=df_temp_polar.apply(lambda x: x['f1']*x['support'],axis=1)
df_temp_raw['ok?'] = df_temp_raw.apply(lambda x: x['predict']==x['label'],axis=1)


/home/aiuser/.pyenv/versions/3.6.6/envs/sk-torch/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [225]:
## 计算加权F1
f1_pos = df_temp_polar[df_temp_polar.polarity==5]['weight_f1'].sum()/df_temp_polar[df_temp_polar.polarity==5]['support'].sum()
f1_neg = df_temp_polar[df_temp_polar.polarity==1]['weight_f1'].sum()/df_temp_polar[df_temp_polar.polarity==1]['support'].sum()

f1_pos, f1_neg



(0.887772210516138, 0.7690163325516203)

In [227]:
df_temp_polar[df_temp_polar.polarity==5].sort_values('f1')

,prec,recall,f1,support,subtype,polarity,weight_f1
3,0.400000,0.571429,0.470588,7.0,Fat Granule,5,3.294118
3,0.701149,0.655914,0.677778,93.0,Whitening,5,63.033333
3,0.863636,0.833333,0.848214,114.0,Irritation,5,96.696429
3,0.945122,0.880682,0.911765,176.0,Smell,5,160.470588
2,0.922652,0.932961,0.927778,179.0,Greasy,5,166.072222
3,0.919192,0.968085,0.943005,282.0,Moisturization,5,265.927461


In [ ]:
df_temp_raw.to_csv(f'/data/projects/bert_pytorch/{data_dir}/polar_raw_results_mod_punc.csv', index=False)

In [ ]:
df_temp_polar.to_csv(f'/data/projects/bert_pytorch/{data_dir}/asop_mtl_polar_bert_ep5.csv', index=False)

## 统计情感词极性冲突的情况

In [ ]:
import math
from collections import defaultdict

stat_op_confict = defaultdict(dict)

for _, line in df_temp_raw.iterrows():
    
    kw = line['op_pred'] or 'null'
    polar = line['predict'] or 'null'
    stat_op_confict[kw][polar] = stat_op_confict[kw].get(polar,0)+1

idx = 0
for k, v in stat_op_confict.items():
    total = sum(v.values())
#     for i,j in v.items():
#         v[i] = round(j/total*100,3)
    if len(v) > 1:
        print(idx,k,v)
        idx+=1
print(f'共有情感词{len(stat_op_confict)}, 其中存在极性冲突{idx}, 占比{idx/len(stat_op_confict)*100}')

## 计算指标(single)

各个subtype分别训练阅读理解模型

In [6]:
from sklearn.metrics import precision_recall_fscore_support
# from utils_squad_multi_plus import read_ecom_examples as cc
# from utils_squad_multi_plus import TRANS_SUBTYPE
from utils_skincare import read_ecom_examples as cc
from utils_skincare import TRANS_SUBTYPE

import os
import numpy as np
import pandas as pd

experiment_name = 'skincare_patch'
runs_name = 'all_in_one_ground'
run_name = f'{runs_name}/checkpoint-4200'  # 有时需要加上checkpoint后缀
data_path = f'/data/projects/bert_pytorch/{experiment_name}'

import os
import json
import pandas as pd

ret = {}
ret_op = {}
ret_polar = {}

dirlist = os.listdir(data_path)
# dirlist = ['Moisturization']
for filename in dirlist:
    subtype = filename.replace('.','/').replace('_'," ")
    if subtype in TRANS_SUBTYPE:
        try:
            print(f'try to assess {subtype}')
            temp = !python evaluate_ecom_asop.py -st aspect -sd skincare -rn {run_name} -tn {experiment_name} {filename} |tail -n 15
            ret[subtype] = json.loads(''.join(temp))
            temp = !python evaluate_ecom_asop.py -st op -sd skincare -rn {run_name} -tn {experiment_name}  {filename} |tail -n 15
            ret_op[subtype] = json.loads(''.join(temp))
#             temp = !python evaluate_ecom_asop.py -st polar  -sd skincare -rn {run_name} -tn {experiment_name} -pl {filename} |tail -n 15
#             ret_polar[subtype] = json.loads(''.join(temp))
        except Exception as e:
            print('err', subtype, e)

data = []     
for k in ret.keys():
    data.append(
        (   k,
            ret[k]['exact'], ret[k]['f1'], ret[k]['total'], 
            ret[k]['NoAns_exact'], ret[k]['NoAns_f1'],ret[k]['NoAns_total'], 
            ret_op[k]['exact'], ret_op[k]['f1'], ret_op[k]['total'], 
            ret_op[k]['NoAns_exact'], ret_op[k]['NoAns_f1'],ret_op[k]['NoAns_total'], 
#             ret_polar[k]['exact'], ret_polar[k]['f1'], ret_polar[k]['total'], 
#             ret_polar[k]['NoAns_exact'], ret_polar[k]['NoAns_f1'],ret_polar[k]['NoAns_total'], 
        )
    )

aspect_columns = ['exact','f1','total','NoAns_exact','NoAns_f1','NoAns_total']
opinions_columns = list(map(lambda x: 'op_'+x, aspect_columns))
polar_columns = list(map(lambda x: 'polar_'+x, aspect_columns))

# df_metric = pd.DataFrame(data, columns =['subtype'] + aspect_columns+opinions_columns+polar_columns)
df_metric = pd.DataFrame(data, columns =['subtype'] + aspect_columns+opinions_columns)

df_metric.sort_values('f1', ascending=True, inplace=True)
# df_metric.to_csv(os.path.join(data_path, 'metric_asop_mtl.csv'), index=False)
# df_metric[['subtype','exact','f1','total', 'op_exact','op_f1','op_total', 'polar_exact','polar_f1','polar_total']]
df_metric[['subtype','exact','f1','total', 'op_exact','op_f1','op_total']]



try to assess Whitening
['{', '  "exact": 76.67844522968198,', '  "f1": 86.99927085086091,', '  "total": 283,', '  "HasAns_exact": 61.8421052631579,', '  "HasAns_f1": 81.05785296574774,', '  "HasAns_total": 152,', '  "NoAns_exact": 93.89312977099236,', '  "NoAns_f1": 93.89312977099236,', '  "NoAns_total": 131,', '  "best_exact": 77.03180212014134,', '  "best_exact_thresh": -2.5290756225585938,', '  "best_f1": 87.35262774132026,', '  "best_f1_thresh": -1.942481517791748', '}']
try to assess Smell
['{', '  "exact": 87.22943722943722,', '  "f1": 93.25429861144148,', '  "total": 462,', '  "HasAns_exact": 75.81967213114754,', '  "HasAns_f1": 87.22740146920478,', '  "HasAns_total": 244,', '  "NoAns_exact": 100.0,', '  "NoAns_f1": 100.0,', '  "NoAns_total": 218,', '  "best_exact": 87.22943722943722,', '  "best_exact_thresh": -0.056565284729003906,', '  "best_f1": 93.25429861144148,', '  "best_f1_thresh": -0.056565284729003906', '}']
try to assess Irritation
['{', '  "exact": 81.67420814479638

,subtype,exact,f1,total,op_exact,op_f1,op_total
0,Whitening,76.678445,86.999271,283,71.024735,81.083467,283
5,Moisturization,82.695507,88.661242,601,79.367720,85.134400,601
2,Irritation,81.674208,89.374608,442,77.375566,85.163971,442
3,Greasy,85.138004,91.509784,471,87.685775,91.819509,471
1,Smell,87.229437,93.254299,462,83.766234,88.198628,462
4,Fat Granule,96.153846,97.202797,26,57.692308,71.249178,26


In [63]:
### 结果写入metric
## 将加权F1写入mlflow文件
import mlflow
from  mlflow.tracking import MlflowClient


mlflow.set_tracking_uri('http://127.0.0.1:9001')

runs_name = 'smell_1'

client = MlflowClient()
experiments = client.list_experiments() # returns a list of mlflow.entities.Experiment
experiment = list(filter(lambda x:x.name==experiment_name, experiments))
target_run = client.search_runs(experiment[0].experiment_id, f"tag.mlflow.runName='{runs_name}' and tag.mlflow.user='ymai'")
# if len(target_run) > 0:
#     for _, line in df_metric.iterrows():
#         sub = line['subtype']
#         f1 = line['f1']
#         op_f1 = line['op_f1']
#         client.log_metric(target_run[0].info.run_id, f'{sub}_f1', f1)
#         client.log_metric(target_run[0].info.run_id, f'{sub}_opf1', op_f1)
#         print('write metric succeed...')
target_run

[<Run: data=<RunData: metrics={'training_loss': 0.7133065998531203}, params={'learning_rate': '2e-05',
  'max_seq_length': '256',
  'num_train_epochs': '4.0',
  'per_gpu_train_batch_size': '16',
  'train_sample_ratio': '1.0'}, tags={'mlflow.runName': 'smell_1',
  'mlflow.source.git.commit': '762abf82ce46134e12f8158697e92eed8bfef758',
  'mlflow.source.name': '../examples/run_skincare_v2.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'ymai',
  'output_dir': '/data/projects/bert_pytorch//skincare_op_out/smell_1'}>, info=<RunInfo: artifact_uri='/data/projects/mlruns/2/f6b60853a36946aea712116a42a17073/artifacts', end_time=1567661669252, experiment_id='2', lifecycle_stage='active', run_id='f6b60853a36946aea712116a42a17073', run_uuid='f6b60853a36946aea712116a42a17073', start_time=1567661398126, status='FINISHED', user_id='ymai'>>]

In [229]:
# ep5
df_old = df_metric[['subtype','exact','f1','op_exact','op_f1']]
df_old

,subtype,exact,f1,op_exact,op_f1
5,Whitening,77.031802,86.613209,72.084806,82.009863
2,Irritation,79.864253,88.628954,77.828054,85.047052
0,Moisturization,83.693844,89.350809,80.698835,86.632476
3,Greasy,84.925690,91.891473,88.959660,92.490295
1,Smell,87.012987,93.065881,83.766234,89.098859
4,Fat Granule,100.000000,100.000000,69.230769,76.752137


## 计算指标（multi）
各个subtype训练统一的阅读理解模型

In [ ]:
data_path = '/data/projects/bert_pytorch/ecom_aspect_bak'

import os
import json
import pandas
from utils_squad_op import TRANS_SUBTYPE

ret = {}
ret_op = {}

for filename in os.listdir(data_path):
    subtype = filename.replace('.','/').replace('_'," ")
    if subtype in TRANS_SUBTYPE:
        try:
            print(f'try to metric {subtype}')
            temp = !python evaluate_ecom_v2.py -m -s {filename} all_in_one_nop |tail -n 15
            ret[subtype] = json.loads(''.join(temp))
#             temp = !python evaluate_ecom_v2.py -op -m -s {filename} all_in_one |tail -n 15
#             ret_op[subtype] = json.loads(''.join(temp))
        except:
            print('err', subtype)

In [ ]:
data = []     
for k in ret.keys():
    data.append(
        (   k,
            ret[k]['exact'], ret[k]['f1'], ret[k]['total'], 
            ret[k]['NoAns_exact'], ret[k]['NoAns_f1'],ret[k]['NoAns_total'], 
#             ret_op[k]['exact'], ret_op[k]['f1'], ret_op[k]['total'], 
#             ret_op[k]['NoAns_exact'], ret_op[k]['NoAns_f1'],ret_op[k]['NoAns_total'], 
        )
    )

aspect_columns = ['exact','f1','total','NoAns_exact','NoAns_f1','NoAns_total']
opinions_columns = list(map(lambda x: 'op_'+x, aspect_columns))

df_metric_all = pd.DataFrame(data, columns =['subtype'] + aspect_columns)
df_metric_all.sort_values('exact', ascending=False, inplace=True)
df_metric_all.to_csv(os.path.join(data_path, 'metric_as_all.csv'), index=False)


In [ ]:
df_metric_all.sort_values([ 'f1'], ascending=False)

## 两种指标具体差异

In [ ]:
file_path ='/data/projects/bert_pytorch/ecom_aspect_bak/raw_answer/'

m = []
with open(os.path.join(file_path, 'package_material_byall.json'), 'r') as f:
    for line in f:
        m.append(json.loads(line))
        


print(m[0])
idx = 0 
for k, v in m[1].items():
    if v['label'] != "":
        idx += 1
        print(idx, v['text'])
        label, pred = v['label'], v['pred']
        print(f'<{label}>, <{pred}>')

In [ ]:
file_path ='/data/projects/bert_pytorch/ecom_aspect_bak/raw_answer/'

m = []
with open(os.path.join(file_path, 'package_material.json'), 'r') as f:
    for line in f:
        m.append(json.loads(line))
        
print(m[0])
idx = 0 
for k, v in m[1].items():
    if v['label'] != "":
        idx += 1
        print(idx, v['text'])
        label, pred = v['label'], v['pred']
        print(f'<{label}>, <{pred}>')

##  NER 语料准备

需要重新改写一下，所句子中出现的所有特征词，情感词统统找出来.

In [ ]:
import re
def find_positions(text, aspect_terms):    
    ret = []
    for term in aspect_terms:
        re_exp = term.strip(' ').replace('  ', ' ').replace(' ', '.{0,20}?')
        ret += [(i.start(), i.end()) for i in re.finditer(re_exp, text)]
    return ret

In [ ]:
raw_data = []

with open('/data/projects/bert_pytorch/ecom_aspect_bak/general.json', 'r') as c:
    for line in c:
        jl = json.loads(line)
        text = jl['text']
        pos = ['N'] * len(text)
        for op in jl['opinions']:
            ret = find_positions(text, op['aspectTerm'])
            for start, end in ret:
                if start != -2:
                    for i in range(start, end):
                        pos[i] = 'A'
                    pos[start] = 'B-A'
            ret = find_positions(text, op['opinionTerm'])
            for start, end in ret:
                if start != -2:
                    for i in range(start, end):
                        if pos[i] == 'N':
                            pos[i] = 'A'
                    if start == 0 or pos[start-1] == 'N':
                        pos[start] = 'B-A'
                        

        raw_data.append({
            'text': text,
            'pos': pos,
        })

In [ ]:
from random import shuffle

shuffle(raw_data)

test_file  = '/home/aiuser/projects/changshuangkai/BERT-BER/data/ecom_v2/test_bio'
train_file  = '/home/aiuser/projects/changshuangkai/BERT-BER/data/ecom_v2/train_bio'

split = int(0.8*len(raw_data))
train_raw_data = raw_data[:split]
test_raw_data = raw_data[split:]

def write_ner_data(file_path, data):
    with open(file_path,'w') as c:
        for i in data:
            for token, tag in zip(i['text'], i['pos']):
                c.write(f'{token} {tag}')
                c.write('\n')
            c.write('\n')


In [ ]:
write_ner_data(train_file, train_raw_data)

In [ ]:
write_ner_data(test_file, test_raw_data)

## 多任务训练

### 两个subtype的阅读理解一起训练

In [ ]:
data_dir = '/data/projects/bert_pytorch/ecom_aspect_mtl/'
for dirname in os.listdir(data_dir):
    subtype = dirname.replace('.','/').replace('_'," ")
    if subtype in TRANS_SUBTYPE:
        print('eval', subtype)
        !../script/run_ecom_multi_eval.sh {dirname}
    

### 集成情感极性

#### 拆分情感极性数据集

In [8]:
import json
from utils_skincare import find_positions
from data_preprocess import convert_text

json_path = '/data/projects/bert_pytorch/ecom_asop_mtl_opti_sample_v2/general.json'
polar_corpus = []

with open(json_path, 'r') as cc:
    for line in cc:
        jl = json.loads(line)
        text = jl['text']
        clear_text = convert_text(text)
        for op in jl['opinions']:
            start, end = find_positions(clear_text, op['opinionTerm'])
            if start != -2:
                subtype = op['aspectSubtype']
                polar = op['polarity']
                term = clear_text[start:end]
                polar_corpus.append((term, clear_text, polar, subtype))
        

In [9]:
import pandas as pd
df_polar = pd.DataFrame(polar_corpus ,columns=['opterm','textb','label','subtype'])
len(df_polar)

126035

In [10]:
import os
from  utils_squad_multi_plus import TRANS_SUBTYPE
ratio = 0.8
corpus_dir = '/data/projects/bert_pytorch/ecom_asop_mtl_opti_sample_v2/'
# corpus_dir = []
for dirname in os.listdir(corpus_dir):
    output_dir = os.path.join(corpus_dir, dirname)
    subtype = dirname.replace('.', '/').replace('_'," ")
    if subtype not in TRANS_SUBTYPE:
        continue
    print(dirname)
    temp  = df_polar[df_polar.subtype == subtype]
    temp = temp.sample(frac=1)
    train_limit = int(ratio * len(temp))
    temp[:train_limit].to_csv(os.path.join(output_dir, 'train.csv'), index=False)
    temp[train_limit:].to_csv(os.path.join(output_dir, 'dev.csv'), index=False)


Package_Material
Package_Design
Logistics_Speed
Logistics_Package
Wrong_Delivery
Promotion
WOM
Price_Sensitivity
Inventory
Pick-up_Speed
Package_Cleanliness
Package_Printing
New_User
Package_Integrity
Price_Satisfaction
Logistics_Damage
Brand_Equity
Logistics_Fee
Expiration_Date
Return_Exchange
Package_General
Logistics_Service
Logistics_Company
Fake_Concern
Loyalty
Shop.Customer_Service


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

data_dir = '/data/projects/bert_pytorch/ecom_aspect_mtl_polar_out/'
subtype = 'Return_Exchange'
labels = []
preds = []
with open(os.path.join(data_dir, subtype, 'polarity_preds_.csv'), 'r') as c:
    for line in c:
        try:
#             print(temp)
            temp = line.strip('\n').rsplit(',',6)
            labels.append(temp[2])
            preds.append(temp[3])
        except:
            pass
          
metric = precision_recall_fscore_support(labels, preds)
pd.DataFrame(np.array(metric).T)

In [ ]:
from utils_squad_polar import read_ecom_examples, read_multi_examples

data_dir = '/data/projects/bert_pytorch/ecom_aspect_mtl_polar/'

ee = read_multi_examples(data_dir)

#### 读取情感极性的F1

##### 多任务训练结果

In [48]:
import os
import pandas as pd
import json

SUBTYPE_DICT = json.load(open('SUBTYPE.json', 'r'))
TRANS_SUBTYPE = SUBTYPE_DICT['skincare']


experiment_name = 'skincare_op'
runs_name = 'smell_028'
result_dir = f'/data/projects/bert_pytorch/{experiment_name}_out/{runs_name}'
# result_dir = []

polarity_map = {
    0:1,
    1:3,
    2:5
}

polar_metric= []

for dirname in os.listdir(result_dir):
    output_dir = os.path.join(result_dir, dirname)
    subtype = dirname.replace('.', '/').replace('_'," ")
    if subtype not in TRANS_SUBTYPE:
        continue
    print(subtype)
    json_path = os.path.join(result_dir, dirname, 'eval_results.txt')
    with open(json_path, 'r') as cc:
        lines = cc.read().splitlines()
        f1, prec, recall, support = [json.loads(item.split('=')[-1]) for item in lines[1:]]
        for idx, m in enumerate(zip(f1, prec, recall, support)):  
            polar_metric.append((
                m[1], m[2], m[0], m[3], subtype, polarity_map[idx]
            ))
df_polar_m = pd.DataFrame(polar_metric, columns=['precision', 'recall', 'f1', 'sample_num', 'subtype', 'polarity'])
df_polar_m['weight_f1'] = df_polar_m.apply(lambda x:x['sample_num']*x['f1'],axis=1)

Smell


In [49]:
## 计算加权F1
f1_pos = df_polar_m[df_polar_m.polarity==5]['weight_f1'].sum()/df_polar_m[df_polar_m.polarity==5]['sample_num'].sum()
f1_neg = df_polar_m[df_polar_m.polarity==1]['weight_f1'].sum()/df_polar_m[df_polar_m.polarity==1]['sample_num'].sum()

f1_pos, f1_neg

(0.9645776566757494, 0.9075630252100839)

In [50]:
## 将加权F1写入mlflow文件
import mlflow
from  mlflow.tracking import MlflowClient


mlflow.set_tracking_uri('http://127.0.0.1:9001')


client = MlflowClient()
experiments = client.list_experiments() # returns a list of mlflow.entities.Experiment
experiment = list(filter(lambda x:x.name==experiment_name, experiments))
target_run = client.search_runs(experiment[0].experiment_id, f"tag.mlflow.runName='{runs_name}'")

if len(target_run) > 0:
    client.log_metric(target_run[0].info.run_id, 'pos_polar_aver', f1_pos)
    client.log_metric(target_run[0].info.run_id, 'neg_polar_aver', f1_neg)
    print('write metric succeed...')


write metric succeed...


In [47]:
runs_name

'ml_test_ep5/checkpoint-4200'

In [265]:
df_polar_m[df_polar_m.polarity==5].sort_values('f1', ascending=True)

,precision,recall,f1,sample_num,subtype,polarity,weight_f1
14,0.875000,1.000000,0.933333,7,Fat Granule,5,6.533333
8,0.968000,0.945312,0.956522,128,Irritation,5,122.434783
5,0.952128,0.988950,0.970190,181,Smell,5,175.604336
17,0.989362,0.968750,0.978947,96,Whitening,5,93.978947
11,0.977778,0.988764,0.983240,178,Greasy,5,175.016760
2,0.984962,0.992424,0.988679,264,Moisturization,5,261.011321


In [246]:
df_polar_m.sort_values(by=['sample_num', 'subtype','polarity'], inplace=True)
df_polar_m.to_csv('/data/projects/bert_pytorch/skincare_op/polar_metric_mtl.csv', index=False)

##### 单一任务训练结果

In [ ]:
df_raw = pd.read_csv('/data/projects/bert_pytorch/ecom_output/label_3_polar/eval_raw.csv',header=None)
df_corpus = pd.read_csv('/data/projects/bert_pytorch/ecom/label_3_polar/dev.csv')

In [ ]:
df_corpus['real'] = df_raw[1]
df_corpus['pred'] = df_raw[0]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
raws = []
for k in TRANS_SUBTYPE.keys():
#     print(k)
    preds = df_corpus[df_corpus.subtype==k]['pred']
    real = df_corpus[df_corpus.subtype==k]['real']
    prec, recall, f1, support = precision_recall_fscore_support(real, preds)
    for idx, metric in enumerate(zip(prec, recall, f1, support)):
        raws.append((
            *metric, k, polarity_map[idx]
        ))
df_polar_m1 = pd.DataFrame(raws, columns=['precision', 'recall', 'f1', 'sample_num', 'subtype', 'polarity'])

In [ ]:
df_polar_m1.sort_values(by=['sample_num', 'subtype','polarity'], inplace=True)
df_polar_m1.to_csv('/data/projects/bert_pytorch/ecom/label_3_polar/polar_metric.csv', index=False)

In [ ]:
df_polar_m1[df_polar_m1.polarity==1].sort_values('sample_num')


#### 极性预测详细结果生成

In [262]:
task_name= 'skincare_op'
runs_name = 'all_in_one/checkpoint-4200'

label_map = {
    1:3,
    0:1,
    2:5
}
polar_dir = os.path.join('/data/projects/bert_pytorch/', task_name)
polar_out_dir = os.path.join('/data/projects/bert_pytorch/', f'{task_name}_out', runs_name)

df_polar_raw = pd.DataFrame()
for dirname in os.listdir(polar_dir):
    subtype = dirname.replace('.', '/').replace('_'," ")
    if subtype not in TRANS_SUBTYPE:
        continue 
    real_file = os.path.join(polar_dir, dirname,'dev.csv')
    df_tmp = pd.read_csv(real_file)
    
    pred_file = os.path.join(polar_out_dir, dirname, 'eval_raw.csv')
    df_temp_pred = pd.read_csv(pred_file, header=None)
    df_tmp['predict'] = df_temp_pred[0].apply(lambda x: label_map[x])
    df_polar_raw = pd.concat([df_polar_raw, df_tmp])
    
df_polar_raw.to_csv(os.path.join(polar_dir, 'polar_raw_results_0903.csv'), index=False)   

In [266]:
import math
from collections import defaultdict

stat_op_confict = defaultdict(dict)

for _, line in df_polar_raw.iterrows():
    kw = line['opterm'] or 'null'
    polar = line['predict'] or 'null'
    stat_op_confict[kw][polar] = stat_op_confict[kw].get(polar,0)+1

idx = 0
for k, v in stat_op_confict.items():
    total = sum(v.values())
#     for i,j in v.items():
#         v[i] = round(j/total*100,3)
    if len(v) > 1:
        print(idx,k,v)
        idx+=1
print(f'共有情感词{len(stat_op_confict)}, 其中存在极性冲突{idx}, 占比{idx/len(stat_op_confict)*100}')

0 滋润 {5: 68, 1: 1}
1 还行 {5: 4, 1: 1}
2 耐受 {5: 4, 1: 1}
3 下去 {5: 1, 3: 1}
4 不油 {5: 10, 1: 1}
5 油腻 {1: 9, 5: 1}
6 控油 {5: 4, 1: 1}
共有情感词540, 其中存在极性冲突7, 占比1.2962962962962963


### span extraction

In [11]:
x=torch.randn(3,10)
x.size()


torch.Size([3, 10])

## 读取runs 参数

In [196]:
train_density_dict = defaultdict(dict)
train_op_density_dict = defaultdict(dict)
for fea in feas:
    if fea.is_impossible == False:
        asp_word =''.join(fea.tokens[fea.start_position:fea.end_position+1])
        train_density_dict['kk'][asp_word] = train_density_dict['kk'].get(asp_word,0) + 1
    if fea.is_op_impossible == False:
        op_word=''.join(fea.tokens[fea.op_start_position:fea.op_end_position+1])
        train_op_density_dict['kk'][op_word] =train_op_density_dict['kk'].get(op_word,0) + 1

dens=len(train_density_dict['kk'])/sum(list(train_density_dict['kk'].values()))
op_dens=len(train_op_density_dict['kk'])/sum(list(train_op_density_dict['kk'].values()))
dens, op_dens

(0.28180354267310787, 0.5824915824915825)

## 统计语料密度
- 各个subtype 关键词数n_kw和语料条数n的关系，用来衡量一个subtype的复杂度
> 初步的猜想是希望 
> - n_kw/n越小越好, 说明有充分语料信息来学习？？
> - 不同的语料密度对训练数据量的要求是一致的吗？？


- 随着采样的进行，高频词更容易被踩到，这样此时词语变得更加分散了，密度提升了。 
- 标注的过程正好相反，是一个密度降低的过程
    - 新词出现本身是一个快到慢的过程，意味着密度是可以逐渐下降的，那么何时收敛
    - 新词出现速度和样本量增长达到稳态了
- 密度越小测试集指标一定会不断提升吗？
    - 还有一个判定非相关的问题。


### skincare语料统计

- **Moisturization**
{'语料大小': 1268, '情感词': 311, '特征词': 137, '冲突词': 0, '情感密度': 0.24526813880126183, '特征密度': 0.10804416403785488}
- **Smell**
{'语料大小': 974, '情感词': 273, '特征词': 157, '冲突词': 0, '情感密度': 0.2802874743326489, '特征密度': 0.16119096509240247}
- **Irritation**
{'语料大小': 930, '情感词': 336, '特征词': 131, '冲突词': 0, '情感密度': 0.36129032258064514, '特征密度': 0.14086021505376345}
- **Greasy**
{'语料大小': 1008, '情感词': 251, '特征词': 146, '冲突词': 0, '情感密度': 0.2490079365079365, '特征密度': 0.14484126984126985}
- **Fat Granule**
{'语料大小': 53, '情感词': 39, '特征词': 3, '冲突词': 0, '情感密度': 0.7358490566037735, '特征密度': 0.05660377358490566}
- **Whitening**
{'语料大小': 587, '情感词': 362, '特征词': 159, '冲突词': 0, '情感密度': 0.616695059625213, '特征密度': 0.2708688245315162}
---
- 情感有难到易
['Fat Granule', 'Whitening', 'Irritation', 'Smell', 'Greasy', 'Moisturization']
- 特征有难到易
['Whitening', 'Smell', 'Greasy', 'Irritation', 'Moisturization', 'Fat Granule']

### general语料统计

- **Package Material**
{'语料大小': 48, '情感词': 45, '特征词': 17, '冲突词': 0, '情感密度': 0.9375, '特征密度': 0.3541666666666667}
- **Package Design**
{'语料大小': 2677, '情感词': 543, '特征词': 456, '冲突词': 0, '情感密度': 0.20283899887934254, '特征密度': 0.17033993276055287}
- **Logistics Speed**
{'语料大小': 7825, '情感词': 1340, '特征词': 1065, '冲突词': 0, '情感密度': 0.17124600638977636, '特征密度': 0.13610223642172525}
- **Logistics Package**
{'语料大小': 5483, '情感词': 1186, '特征词': 876, '冲突词': 0, '情感密度': 0.21630494254969906, '特征密度': 0.1597665511581251}
- **Wrong Delivery**
{'语料大小': 1016, '情感词': 375, '特征词': 259, '冲突词': 0, '情感密度': 0.3690944881889764, '特征密度': 0.2549212598425197}
- **Promotion**
{'语料大小': 10239, '情感词': 3056, '特征词': 857, '冲突词': 0, '情感密度': 0.2984666471335091, '特征密度': 0.083699580037113}
- **WOM**
{'语料大小': 996, '情感词': 52, '特征词': 111, '冲突词': 0, '情感密度': 0.05220883534136546, '特征密度': 0.11144578313253012}
- **Price Sensitivity**
{'语料大小': 3956, '情感词': 656, '特征词': 336, '冲突词': 1, '情感密度': 0.16582406471183014, '特征密度': 0.08493427704752275}
- **Inventory**
{'语料大小': 430, '情感词': 98, '特征词': 25, '冲突词': 0, '情感密度': 0.22790697674418606, '特征密度': 0.05813953488372093}
- **Pick-up Speed**
{'语料大小': 2939, '情感词': 486, '特征词': 248, '冲突词': 0, '情感密度': 0.1653623681524328, '特征密度': 0.0843824430078258}
- **Package Cleanliness**
{'语料大小': 440, '情感词': 121, '特征词': 252, '冲突词': 0, '情感密度': 0.275, '特征密度': 0.5727272727272728}
- **Package Printing**
{'语料大小': 61, '情感词': 46, '特征词': 39, '冲突词': 0, '情感密度': 0.7540983606557377, '特征密度': 0.639344262295082}
- **New User**
{'语料大小': 1071, '情感词': 416, '特征词': 77, '冲突词': 0, '情感密度': 0.388422035480859, '特征密度': 0.0718954248366013}
- **Package Integrity**
{'语料大小': 2111, '情感词': 323, '特征词': 641, '冲突词': 0, '情感密度': 0.15300805305542398, '特征密度': 0.3036475603979157}
- **Price Satisfaction**
{'语料大小': 12180, '情感词': 1366, '特征词': 371, '冲突词': 1, '情感密度': 0.11215106732348111, '特征密度': 0.03045977011494253}
- **Logistics Damage**
{'语料大小': 5292, '情感词': 650, '特征词': 1108, '冲突词': 0, '情感密度': 0.12282690854119425, '特征密度': 0.20937263794406652}
- **Brand Equity**
{'语料大小': 5406, '情感词': 1755, '特征词': 189, '冲突词': 0, '情感密度': 0.3246392896781354, '特征密度': 0.03496115427302997}
- **Logistics Fee**
{'语料大小': 1053, '情感词': 516, '特征词': 123, '冲突词': 1, '情感密度': 0.49002849002849, '特征密度': 0.1168091168091168}
- **Expiration Date**
{'语料大小': 2265, '情感词': 534, '特征词': 233, '冲突词': 0, '情感密度': 0.23576158940397351, '特征密度': 0.10286975717439294}
- **Return Exchange**
{'语料大小': 1447, '情感词': 754, '特征词': 112, '冲突词': 0, '情感密度': 0.5210780926053905, '特征密度': 0.07740152038700761}
- **Package General**
{'语料大小': 1825, '情感词': 632, '特征词': 600, '冲突词': 0, '情感密度': 0.3463013698630137, '特征密度': 0.3287671232876712}
- **Logistics Service**
{'语料大小': 5376, '情感词': 1330, '特征词': 1019, '冲突词': 0, '情感密度': 0.24739583333333334, '特征密度': 0.18954613095238096}
- **Logistics Company**
{'语料大小': 1403, '情感词': 421, '特征词': 36, '冲突词': 0, '情感密度': 0.3000712758374911, '特征密度': 0.02565930149679259}
- **Fake Concern**
{'语料大小': 5619, '情感词': 1614, '特征词': 784, '冲突词': 1, '情感密度': 0.2872397223705286, '特征密度': 0.1395266061576793}
- **Loyalty**
{'语料大小': 13088, '情感词': 2563, '特征词': 1456, '冲突词': 0, '情感密度': 0.19582823960880197, '特征密度': 0.11124694376528117}
- **Shop/Customer Service**
{'语料大小': 6574, '情感词': 2403, '特征词': 1908, '冲突词': 0, '情感密度': 0.36553087922117433, '特征密度': 0.2902342561606328}
---
- 情感有难到易
['Package Material', 'Package Printing', 'Return Exchange', 'Logistics Fee', 'New User', 'Wrong Delivery', 'Shop/Customer Service', 'Package General', 'Brand Equity', 'Logistics Company', 'Promotion', 'Fake Concern', 'Package Cleanliness', 'Logistics Service', 'Expiration Date', 'Inventory', 'Logistics Package', 'Package Design', 'Loyalty', 'Logistics Speed', 'Price Sensitivity', 'Pick-up Speed', 'Package Integrity', 'Logistics Damage', 'Price Satisfaction', 'WOM']
- 特征有难到易
['Package Printing', 'Package Cleanliness', 'Package Material', 'Package General', 'Package Integrity', 'Shop/Customer Service', 'Wrong Delivery', 'Logistics Damage', 'Logistics Service', 'Package Design', 'Logistics Package', 'Fake Concern', 'Logistics Speed', 'Logistics Fee', 'WOM', 'Loyalty', 'Expiration Date', 'Price Sensitivity', 'Pick-up Speed', 'Promotion', 'Return Exchange', 'New User', 'Inventory', 'Brand Equity', 'Price Satisfaction', 'Logistics Company']


In [17]:
from utils_squad_multi_plus import TRANS_SUBTYPE
# from utils_skincare import TRANS_SUBTYPE

import math
from collections import defaultdict

experiment_name = 'ecom_asop_mtl_opti_sample_v2'
# runs_name = 'smell_1'
# run_name = f'{runs_name}'  # 有时需要加上checkpoint后缀
data_path = f'/data/projects/bert_pytorch/{experiment_name}'
frac=0.85


def generate_corpus_stats(experiment_name, data_path, sublist=TRANS_SUBTYPE, frac=1, verbose=False):
    corpus_stats = defaultdict(dict)
    for dirname in os.listdir(data_path):
        subtype = dirname.replace('.','/').replace('_'," ")
        if subtype not in sublist:
            continue

        stat_op_confict = defaultdict(dict)
        stat_asp = set()
        df_temp_raw = pd.read_csv(os.path.join(data_path, dirname,'dev.csv'))
        df_temp_raw = df_temp_raw.sample(frac=frac)

        idx = 0
        train_json_path = os.path.join(data_path, dirname, 'dev.json')
        train_num = !wc -l {train_json_path}
        threshold = int(frac * int(train_num[0].split()[0]))
#         print(threshold)

        with open(train_json_path, 'r') as c:
            for line in c:
                if random.randint(0,1000) > 1000 * frac:
                    continue
#                 idx += 1
#                 if idx > threshold: break
                jl = json.loads(line)
                for op in jl['opinions']:
                    if op['aspectSubtype'] == subtype:
                        stat_asp.add(op['aspectTerm'][0])
        print(subtype, stat_asp)

        for _, line in df_temp_raw.iterrows():
            kw = line['opterm'] or 'null'
            polar = line['label'] or 'null'
            stat_op_confict[kw][polar] = stat_op_confict[kw].get(polar,0)+1

        idx = 0
        print('打印冲突词', subtype)
        for k, v in stat_op_confict.items():
            total = sum(v.values())

            if len(v) > 1:
                print(idx,k,v)
                idx+=1
        corpus_stats[subtype].update({
            'total': len(df_temp_raw),
            'opinion': len(stat_op_confict),
            'aspect': len(stat_asp),
            'conflict': idx,
            'op_den': len(stat_op_confict)/len(df_temp_raw),
            'asp_den': len(stat_asp)/len(df_temp_raw),
        })


    if verbose:
        for k, v in corpus_stats.items():
            print(f'**{k}**')
            print(v['asp_den'],v['op_den'])

        print('情感有难到易')
        print(sorted(corpus_stats, key=lambda x: corpus_stats[x]['op_den'], reverse=True))
        print('特征有难到易')
        print(sorted(corpus_stats, key=lambda x: corpus_stats[x]['asp_den'], reverse=True))
    return corpus_stats



In [174]:
fracs = [0.1 ,0.3, 0.5, 0.7, 0.75, 0.8, 0.9, 1]
tmp_data = []
for i in range(10,105,5):
    frac = i/100
    r = generate_corpus_stats(experiment_name, data_path,frac=frac)
    tmp_line = [frac]
    for k, v in r.items():
        tmp_line.append(v['asp_den'])
        tmp_line.append(v['op_den'])
        
    tmp_data.append(tmp_line)

cols = [frac]
for k,v in r.items():
    cols.append(f'{k}_den')
    cols.append(f'{k}_op_den')
dff = pd.DataFrame(tmp_data, columns=cols)
dff



0 补了邮费 {5: 1, 1: 1}
0 再优惠点就好 {3: 1, 1: 1}
0 和之前不太一样 {1: 1, 5: 1}
0 补了邮费 {1: 1, 5: 1}
0 和超市价格差不多 {3: 1, 5: 1}
0 再优惠点就好 {3: 1, 1: 1}
0 和之前不太一样 {5: 1, 1: 1}
0 和超市价格差不多 {3: 1, 5: 1}
0 和之前不太一样 {5: 1, 1: 1}
0 补了邮费 {1: 1, 5: 1}
0 和超市价格差不多 {3: 1, 5: 1}
0 补了邮费 {1: 1, 5: 1}
0 和超市价格差不多 {3: 1, 5: 1}
0 再优惠点就好 {3: 1, 1: 1}
0 和之前不太一样 {1: 1, 5: 1}
0 和超市价格差不多 {3: 1, 5: 1}
0 补了邮费 {5: 1, 1: 1}
0 再优惠点就好 {3: 1, 1: 1}
0 和之前不太一样 {1: 1, 5: 1}
0 和超市价格差不多 {5: 2, 3: 1}
0 补了邮费 {1: 1, 5: 1}
0 再优惠点就好 {3: 1, 1: 1}
0 和之前不太一样 {1: 1, 5: 1}
0 和超市价格差不多 {5: 1, 3: 1}
0 补了邮费 {1: 1, 5: 1}
0 和之前不太一样 {5: 1, 1: 1}
0 和超市价格差不多 {5: 2, 3: 1}
0 补了邮费 {5: 1, 1: 1}
0 再优惠点就好 {1: 1, 3: 1}
0 和之前不太一样 {5: 1, 1: 1}


,1.0,Package Cleanliness_den,Package Cleanliness_op_den,Package Material_den,Package Material_op_den,Price Sensitivity_den,Price Sensitivity_op_den,Package Printing_den,Package Printing_op_den,Package Integrity_den,...,Wrong Delivery_den,Wrong Delivery_op_den,Price Satisfaction_den,Price Satisfaction_op_den,Fake Concern_den,Fake Concern_op_den,Brand Equity_den,Brand Equity_op_den,Logistics Service_den,Logistics Service_op_den
0,0.10,0.727273,0.477273,1.800000,1.000000,0.184343,0.287879,2.333333,0.666667,0.540284,...,0.450980,0.617647,0.091133,0.214286,0.339858,0.464413,0.157116,0.521257,0.423792,0.434944
1,0.15,0.772727,0.545455,1.571429,1.000000,0.175379,0.231029,2.000000,0.666667,0.555205,...,0.394737,0.565789,0.079365,0.184455,0.262159,0.417556,0.127004,0.489519,0.336228,0.393300
2,0.20,0.625000,0.397727,0.900000,1.000000,0.155499,0.232617,1.666667,0.916667,0.521327,...,0.413793,0.512315,0.064450,0.177750,0.251779,0.420819,0.106383,0.448659,0.322791,0.381395
3,0.25,0.709091,0.427273,1.083333,1.000000,0.149646,0.240647,1.533333,0.866667,0.431818,...,0.330709,0.484252,0.059442,0.158949,0.226335,0.383630,0.088757,0.434172,0.308780,0.351190
4,0.30,0.689394,0.386364,1.142857,1.000000,0.147430,0.213985,1.500000,0.777778,0.470774,...,0.363934,0.472131,0.052819,0.154078,0.212930,0.370107,0.079531,0.402589,0.293862,0.343459
5,0.35,0.720779,0.370130,0.705882,0.941176,0.126354,0.215162,1.571429,0.857143,0.424899,...,0.351124,0.469101,0.050903,0.143092,0.214032,0.357905,0.071882,0.395877,0.277365,0.325717
6,0.40,0.738636,0.346591,0.842105,0.947368,0.123262,0.193426,1.166667,0.916667,0.433649,...,0.312808,0.431034,0.050903,0.137931,0.213078,0.352313,0.064755,0.386679,0.260000,0.312558
7,0.45,0.737374,0.378788,0.681818,1.000000,0.111236,0.196067,1.222222,0.851852,0.384211,...,0.304158,0.402626,0.046707,0.138113,0.194543,0.347568,0.058364,0.376079,0.255891,0.301364
8,0.50,0.654545,0.309091,0.708333,1.000000,0.109201,0.183013,1.166667,0.866667,0.376894,...,0.332677,0.456693,0.042693,0.130870,0.180071,0.332740,0.056974,0.365520,0.236607,0.289062
9,0.55,0.607438,0.338843,0.615385,1.000000,0.104320,0.179228,1.029412,0.705882,0.367786,...,0.313059,0.436494,0.039857,0.130467,0.180583,0.321683,0.050118,0.356542,0.229625,0.280352


In [177]:
dff.to_csv(os.path.join(data_path, 'sample_density.csv'), index=False)

In [3]:
from data_preprocess import convert_text

tt = convert_text( '这个不错肯定是正品???味道也对，洗完舒服的一批~而且幸运的是拿到了新包装')

## 新词发现功能

In [18]:
import random
experiment_name = 'skincare_op'
runs_name = 'all_in_one'
run_name = f'{runs_name}/checkpoint-4200'  # 有时需要加上checkpoint后缀
data_path = f'/data/projects/bert_pytorch/{experiment_name}'
frac=1

r = generate_corpus_stats(experiment_name, data_path, sublist=['Irritation'], frac=1, verbose=False)



Irritation {'灼烧感', '辣辣的', '小疙瘩', '长痘痘', '眼睛 不舒服', '没有过敏', '耐受', '小红点', '没感到闷痘', '辣眼睛', '粉刺', '很痒', '不良反应', '出斑', '过敏 没有', '刺痛', '不刺激', '长痘', '脸发烫', '不耐受', '出痘', '温和', '说会痛', '长 痘', '刺痛感', '爆痘', '有点痛', '不过敏', '痘痘肌', '刺激', '有点痒', '刺刺的', '疙瘩', '用完长痘长闭口', '灼热感', '起 痘痘', '刺激感', '刺脸', '疼痛', '还痒', '红肿', '有点疼', '火辣辣的', '刺痛掉皮', '大爆痘', '闷痘', '起 痘', '脸上 痒', '起痘', '又疼', '痘痘肌用', '不适感', '白色 点点', '毛孔堵', '过敏现象', '长 痘痘', '痘肌', '洗完脸痛', '脸会痒痒', '又氧又疼', '不适', '刚擦上 有点热', '无过敏', '冒痘', '过敏', '脸可痛', '不长痘', '痒痒的', '用着 痒', '起痘痘', '痘痘'}
打印冲突词 Irritation


In [15]:
tt = set(('闭口冒出来', '没伤害', '小红点', '发红', '没过敏', '刺激性', '辣眼睛', '不良反应', '还长 痘', '生痛', '长痘', '烂脸', '火辣辣', '红疙瘩', '刺刺的', '疙瘩', '脸 烂', '刺激感', '火辣辣 痛', '爆豆', '起 痘', '又疼', '洗完 舒适', '长 痘痘', '眼睛周围 痛', '红 肿', '无过敏', '过敏', '不长痘', '长粉刺', '辣辣的', '痘痘', '致痘', '无刺激', '小疙瘩', '没长痘', '起疙瘩', '张 痘痘', '眼睛 肿', '没有过敏', '干疼', '不刺激', '脸上 痒痒', '还长痘', '发痒', '痘痘肌', '刺激', '看痘印说', '疼痛', '火辣辣的', '有一点疼', '有点辣', '痘痘肌用', '不耐反应', '过敏现象', '过敏反应', '不痒了', '痘肌', '有点刺', '耐受性', '满脸痘痘', '刺到眼睛', '耐受', '又痒', '脸 刺痛', '长痘痘', '起红', '会痛', '特别辣', '起 疹子', '痛感', '粉刺', '辣辣的感觉', '脸痛', '有点烧', '刺痛', '出痘', '温和', '长 痘', '爆痘', '会痒', '不过敏', '扎扎的疼', '脸肿', '很疼', '长闭口', '特别疼', '有点疼', '闷痘', '痘痘肌肤', '痒', '起痘', '刺眼', '不适感', '皮肤舒适', '刺痒', '不起痘', '发烫', '不适', '烧灼感', '爆了 痘', '刺眼镜', '起痘痘', '冒痘痘', '有红点', '火辣辣 疼', '冒 痘痘', '有点发热', '皮肤过敏', '柔和', '泛红', '刺痛感', '有点痛', '逗痘', '奇痒无比', '痒痒', '有点痒', '刺疼', '有痘痘', '起 痘痘', '红肿', '没有 过敏', '刺激感都', '发痘痘', '抹了疼', '青春痘', '脸会痒痒', '又氧又疼', '皮肤刺痛', '冒痘', '痘痘长'))

In [19]:
dd = set(('灼烧感', '辣辣的', '小疙瘩', '长痘痘', '眼睛 不舒服', '没有过敏', '耐受', '小红点', '没感到闷痘', '辣眼睛', '粉刺', '很痒', '不良反应', '出斑', '过敏 没有', '刺痛', '不刺激', '长痘', '脸发烫', '不耐受', '出痘', '温和', '说会痛', '长 痘', '刺痛感', '爆痘', '有点痛', '不过敏', '痘痘肌', '刺激', '有点痒', '刺刺的', '疙瘩', '用完长痘长闭口', '灼热感', '起 痘痘', '刺激感', '刺脸', '疼痛', '还痒', '红肿', '有点疼', '火辣辣的', '刺痛掉皮', '大爆痘', '闷痘', '起 痘', '脸上 痒', '起痘', '又疼', '痘痘肌用', '不适感', '白色 点点', '毛孔堵', '过敏现象', '长 痘痘', '痘肌', '洗完脸痛', '脸会痒痒', '又氧又疼', '不适', '刚擦上 有点热', '无过敏', '冒痘', '过敏', '脸可痛', '不长痘', '痒痒的', '用着 痒', '起痘痘', '痘痘'))

In [20]:
dd.difference(tt)

{'不耐受',
 '出斑',
 '刚擦上 有点热',
 '刺痛掉皮',
 '刺脸',
 '大爆痘',
 '很痒',
 '毛孔堵',
 '没感到闷痘',
 '洗完脸痛',
 '灼烧感',
 '灼热感',
 '用完长痘长闭口',
 '用着 痒',
 '痒痒的',
 '白色 点点',
 '眼睛 不舒服',
 '脸上 痒',
 '脸发烫',
 '脸可痛',
 '说会痛',
 '过敏 没有',
 '还痒'}

In [21]:
import torch

aa = torch.load('/data/projects/bert_pytorch/skincare_pa/Greasy/cached_train_pytorch_model.bin_256')



ModuleNotFoundError: No module named 'utils_skincare_v2'